In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
x = torch.randn(4, 3)

In [6]:
x

tensor([[-0.2535, -1.0444,  0.9867],
        [ 1.8119, -0.2154,  0.8950],
        [-0.4492, -3.1261,  0.0776],
        [-1.7039,  1.3526,  0.4245]])

In [7]:
x.T

tensor([[-0.2535,  1.8119, -0.4492, -1.7039],
        [-1.0444, -0.2154, -3.1261,  1.3526],
        [ 0.9867,  0.8950,  0.0776,  0.4245]])

In [12]:
x.view(-1, 2)

tensor([[-0.2535, -1.0444],
        [ 0.9867,  1.8119],
        [-0.2154,  0.8950],
        [-0.4492, -3.1261],
        [ 0.0776, -1.7039],
        [ 1.3526,  0.4245]])

In [13]:
params = list(net.parameters())

In [14]:
len(params)

10

In [15]:
print(params[0].size())

torch.Size([6, 1, 3, 3])


In [16]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0929,  0.0027,  0.0516,  0.0603,  0.0111,  0.0163, -0.0994,  0.0527,
         -0.0353, -0.0371]], grad_fn=<AddmmBackward>)


In [17]:
net.zero_grad()

In [18]:
out.backward(torch.randn(1, 10))

In [25]:
output = net(input)
target = torch.randn(10)

In [26]:
target

tensor([ 0.4540, -0.8687,  0.7808, -0.2444,  0.7461, -0.1560, -0.3715, -0.5996,
         2.1391, -2.9537])

In [27]:
target =target.view(1, -1)

In [28]:
target

tensor([[ 0.4540, -0.8687,  0.7808, -0.2444,  0.7461, -0.1560, -0.3715, -0.5996,
          2.1391, -2.9537]])

In [29]:
criterion = nn.MSELoss()

loss = criterion(output, target)

In [30]:
loss

tensor(1.5987, grad_fn=<MseLossBackward>)

In [34]:
print(loss.grad_fn)

In [35]:
print(loss.grad_fn.next_functions[0][0])

In [36]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [37]:
import torch.optim as optim

In [38]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [39]:
optimizer.zero_grad()

In [40]:
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()